# Quantize pytorch model with 🤗 Accelerate and `bitsandbytes`

## Running GPT2 on Google Colab

Welcome!

In this notebook, we will learn how to quantize a GTP2 model in 8-bit or in 4-bit and run them for inference. We will use the GPT2 model from minGPT.

Check out this [notebook](https://colab.research.google.com/drive/1VoYNfYDKcKRQRor98Zbf2-9VQTtGJ24k?usp=sharing) if you want to learn how to finetune a quantized model with peft.


## Install dependencies

In [1]:
!git clone https://github.com/karpathy/minGPT.git
!pip install --quiet minGPT/
!pip install --quiet bitsandbytes huggingface_hub
!pip install --quiet git+https://github.com/huggingface/accelerate.git

Cloning into 'minGPT'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 489 (delta 153), reused 145 (delta 145), pack-reused 250 (from 1)
Receiving objects: 100% (489/489), 1.43 MiB | 3.35 MiB/s, done.
Resolving deltas: 100% (268/268), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Import libraries

In [2]:
import torch
from mingpt.model import GPT
from mingpt.bpe import BPETokenizer
from huggingface_hub import snapshot_download
from accelerate import init_empty_weights
from accelerate.utils import load_and_quantize_model, BnbQuantizationConfig
from accelerate import Accelerator

## Tokenizer and weights

In [3]:
# create tokenizer
prompt = "Hello my name is"
tokenizer = BPETokenizer()
x1 = tokenizer(prompt).to(0)

downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json to /root/.cache/mingpt/encoder.json
downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe to /root/.cache/mingpt/vocab.bpe


I've stored the weights of the GPT2 model in huggingface hub.

In [4]:
# download weights from huggingface hub
weights_location = snapshot_download(repo_id="marcsun13/gpt2-xl-linear-sharded")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/871M [00:00<?, ?B/s]

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

...

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/972M [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/987M [00:00<?, ?B/s]

pytorch_model-00004-of-00007.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/972M [00:00<?, ?B/s]

## Loading 8-bit model

We instantiate the model under the init_empty_weights context manager in order to load an empty model

In [5]:
model_config = GPT.get_default_config()
model_config.model_type = 'gpt2-xl'
model_config.vocab_size = 50257
model_config.block_size = 1024

# load model on meta device
with init_empty_weights():
  empty_model = GPT(model_config)

number of parameters: 1557.61M


In [6]:
print(empty_model)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1600, out_features=4800, bias=True)
          (c_proj): Linear(in_features=1600, out_features=1600, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=1600, out_features=6400, bias=True)
          (c_proj): Linear(in_features=6400, out_features=1600, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head

We define the quantization configuration that we want and call `load_and_quantize_model` to load the weights and quantize our empty GPT2 model.

In [8]:
# get quantization config
config = BnbQuantizationConfig(load_in_8bit=True, llm_int8_threshold=6)
model_8bit = load_and_quantize_model(empty_model,
                                     bnb_quantization_config = config,
                                     weights_location = weights_location,
                                     device_map="auto")

As you can see, the `nn.Linear` layers are replaced by `bnb.nn.Linear8bitLt` layers. `lm_head` was not replaced in order to ensure stability.

In [9]:
print(model_8bit)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear8bitLt(in_features=1600, out_features=4800, bias=True)
          (c_proj): Linear8bitLt(in_features=1600, out_features=1600, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear8bitLt(in_features=1600, out_features=6400, bias=True)
          (c_proj): Linear8bitLt(in_features=6400, out_features=1600, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_aff

We put the model in evaluation mode and we run the model.

In [10]:
model_8bit.eval()
outputs = model_8bit.generate(x1, max_new_tokens=10, do_sample=False)[0]
print(tokenizer.decode(outputs.cpu().squeeze()))

Hello my name is!!!!!!!!!!


## Saving 8-bit model

::You can save 8-bit model with `save_model` method from `Accelerator()`. Then, you can use these new weights to load your 8-bit model.

In [11]:
accelerate = Accelerator()
new_weights_location = "gpt2-xl-linear-8-bit"
accelerate.save_model(model_8bit, new_weights_location)

## Loading 4-bit model

In [12]:
model_config = GPT.get_default_config()
model_config.model_type = 'gpt2-xl'
model_config.vocab_size = 50257
model_config.block_size = 1024

with init_empty_weights():
  empty_model = GPT(model_config)

number of parameters: 1557.61M


In [13]:
# get quantization config
config = BnbQuantizationConfig(load_in_4bit=True,
                               bnb_4bit_compute_dtype=torch.bfloat16,
                               bnb_4bit_use_double_quant=True,
                               bnb_4bit_quant_type="nf4"
                               )

model_4bit = load_and_quantize_model(empty_model,
                                     bnb_quantization_config = config,
                                     weights_location = weights_location,
                                     device_map="auto")

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1674: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=torc

As you can see, the `nn.Linear` layers are replaced by `bnb.nn.Linear4bit layers`. `lm_head` was not replaced in order to ensure stability.



In [14]:
print(model_4bit)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear4bit(in_features=1600, out_features=4800, bias=True)
          (c_proj): Linear4bit(in_features=1600, out_features=1600, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear4bit(in_features=1600, out_features=6400, bias=True)
          (c_proj): Linear4bit(in_features=6400, out_features=1600, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True

In [15]:
model_4bit.eval()
outputs = model_4bit.generate(x1, max_new_tokens=10, do_sample=False)[0]
print(tokenizer.decode(outputs.cpu().squeeze()))

Hello my name is John. I am a student at the University of


In [18]:
print(torch.cuda.device_count())

1


In [21]:
prop = torch.cuda.get_device_properties(0)
print(prop)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40, uuid=22617769-951a-9350-37a0-65eddefc6e00, L2_cache_size=4MB)
